# Scraping - CLEAR/FGV

## Passo 1: Instalar e importar pacotes necessários

`!pip install selenium`

`!pip install mysql-connector-python`

In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import mysql.connector as mdb
import unicodedata
import re
import glob
import time
from datetime import datetime as dt
#from zipfile import ZipFile
import warnings
warnings.filterwarnings('ignore')

## Passo 2: Realizar o scraping

Abrindo um navegador com a máquina utilizando o webdriver.

In [3]:
driver = webdriver.Chrome()
url = 'https://formosodoaraguaia.megasofttransparencia.com.br/receitas-e-despesas/empenho?faseDoEmpenho=4&etapaDaDespesa=4'
driver.get(url)

Criando duas funções p/ retirar acentos e caracteres especiais fora do padrão.

In [4]:
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError):  
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def text_to_sql(text):
    text = strip_accents(text.lower())
    text = re.sub('[^@$!?&.#0-9a-zA-Z_-]',"_", text)
    text= text.lstrip()
    return text

Construindo uma linha de loop p/ realizar o scraping no site. Devido ao tamanho do conjunto de dados e a consequente carga 
computacional que será demandada para realizar a tarefa exata, optei por realizar a coleta de uma amostra com informações sobre 
empenhos de 24 de maio de 2023 até a data atual. 
Para realizar a tarefa integral, basta alterar o objeto 'end_datetime', no início do código.

In [28]:
end_datetime = dt(2023, 5, 24, 0, 0)
while df.data.tolist()[-1] > end_datetime:
    driver.find_element("xpath", '/html/body/mega-root/div/div/mega-principal/div/mega-empenho/div/mega-formulario-dinamico/form/div[2]/div/mega-fieldset/div[3]/div/mega-table/div/table/tfoot/tr/td/div/div[1]/ngb-pagination/ul/li[8]/a/span').click()
    time.sleep(5)
    l = []
    for i in range(1, 11):
        driver.find_element("xpath", '/html/body/mega-root/div/div/mega-principal/div/mega-empenho/div/mega-formulario-dinamico/form/div[2]/div/mega-fieldset/div[3]/div/mega-table/div/table/tbody[' + str(i) + ']/tr/td[11]/mega-botao/button').click()
        time.sleep(5)
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'lxml')
        chave = soup.find_all('label', attrs={'class' : 'chave'})
        valor = soup.find_all('label', attrs={'class' : 'valor'})
        driver.find_element("xpath", '/html/body/ngb-modal-window/div/div/div[1]/div/mega-botao/button').click()
        f = {}
        for j in range(0, len(chave)):
            f[text_to_sql(strip_accents(chave[j].text.replace(':', '')).lower())] = valor[j].text
        l.append(f)
    dfi = pd.DataFrame(l)
    dfi['data'] = pd.to_datetime(dfi.data)
    df = pd.concat([df, dfi])
    if df.data.tolist()[-1] == end_datetime:
        break
df

,data,fonte,fornecedor,orgao_unidade_gestora,unidade,funcao,subfuncao,programa,acao,dotacao_orcamentaria,...,valor_da_anulacao_da_ordem_de_pagamento,valor_total_do_pagamento,processo_administrativo,modalidade,licitacao_ano,fase,rubrica_despesa,numero_do_contrato,tipo_do_empenho,bem_fornecido_ou_servico_prestado
0,2023-05-31,1.500.1002.000000 - ASPS - Ações e Serviços P...,24.074.470/0001-68/CLINICA MEDICA CAMPOS EIRE...,4 - FUNDO MUN. DE SAUDE FORMOSO DO ARAGUAIA,12 - FUNDO MUNICIPAL DE SAUDE,10 - SAÚDE,302 - ASSISTÊNCIA HOSPITALAR E AMBULATORIAL,10315 - SAÚDE DIREITO DO CIDADÃO,2 - MANTER PRONTO SOCORRO / CAPS,04.12.10.302.10315.2.339039,...,"R$ 0,00","R$ 0,00",722021,19 - Chamamento/Credenciamento Público,12021,Emissão,33903950 - MÉDICOS,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM CRED...
1,2023-05-30,1.500.0000.000000 - Impostos não vinculados,38.480.650/0001-54/CHARLES ALVES PEREIRA 0162...,3 - PREFEITURA MUN. DE FORMOSO DO ARAGUAIA,"3 - SECRETARIA MUNICIPAL DE TURISMO, ESPORTE",27 - DESPORTO E LAZER,812 - DESPORTO COMUNITÁRIO,"10337 - Incentivo ao Esporte, Lazer e Turismo",2 - MANTER ESPAÇOS ESPORTIVOS,03.03.27.812.10337.2.339030,...,"R$ 0,00","R$ 270,00",5972023,99 - NÃO SE APLICA LICITAÇÃO,,Emissão,33903024 - MATERIAL PARA MANUTENCAO DE BENS ...,,Ordinário,EMPENHO EMITIDO PARA OCORRRER DESPESA COM AQU...
2,2023-05-30,1.500.1002.000000 - ASPS - Ações e Serviços P...,07.626.776/0001-60/CIRURGICA SAO FELIPE PRODU...,4 - FUNDO MUN. DE SAUDE FORMOSO DO ARAGUAIA,12 - FUNDO MUNICIPAL DE SAUDE,10 - SAÚDE,301 - ATENÇÃO BÁSICA,10315 - SAÚDE DIREITO DO CIDADÃO,2 - ADQUIRIR MEDICAMENTOS E MATERIAIS PARA U,04.12.10.301.10315.2.339030,...,"R$ 0,00","R$ 16.739,24",962022,20 - ADESÃO À ATA DE REGISTRO DE PREÇOS,222022,Emissão,33903036 - MATERIAL HOSPITALAR,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM ADE...
3,2023-05-26,1.660.0000.000000 - Transferências do FNAS As...,34.223.365/0001-05/L.P. SIQUEIRA,5 - FUNDO MUN. DE ASSISTENCIA SOCIAL - FMAS,13 - FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,8 - ASSISTÊNCIA SOCIAL,244 - ASSISTÊNCIA COMUNITÁRIA,10316 - ASSISTÊNCIA SOCIAL,2 - PROMOVER APRIMORAMENTO GESTÃO DO CADRAST,05.13.08.244.10316.2.339030,...,"R$ 0,00","R$ 2.335,26",14072021,14 - PREGÃO ELETRÔNICO,272021,Emissão,33903024 - MATERIAL PARA MANUTENCAO DE BENS ...,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM AQUI...
4,2023-05-25,1.500.0000.000000 - Impostos não vinculados,25.043.514/0001-55/SECRETARIA DA FAZENDA,3 - PREFEITURA MUN. DE FORMOSO DO ARAGUAIA,9 - SECRETARIA MUNICIPAL DE INFRA-ESTRUTURA,15 - URBANISMO,451 - INFRA-ESTRUTURA URBANA,10312 - INFRA-ESTRUTURA PÚBLICA,2 - MANTER A MALHA VIARIA URBANAS,03.09.15.451.10312.2.339047,...,"R$ 0,00","R$ 72,11",10632023,99 - NÃO SE APLICA LICITAÇÃO,,Emissão,33904710 - TAXAS,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM PAGA...
5,2023-05-25,1.660.0000.000000 - Transferências do FNAS As...,41.930.974/0001-51/P R COELHO ALIMENTOS LTDA,5 - FUNDO MUN. DE ASSISTENCIA SOCIAL - FMAS,13 - FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,8 - ASSISTÊNCIA SOCIAL,244 - ASSISTÊNCIA COMUNITÁRIA,10316 - ASSISTÊNCIA SOCIAL,2 - PROMOVER APRIMORAMENTO GESTÃO DO CADRAST,05.13.08.244.10316.2.339030,...,"R$ 0,00","R$ 4.609,93",10562022,10 - DISPENSA DE LICITAÇÃO,312022,Emissão,33903007 - GENEROS DE ALIMENTACAO,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM AQUI...
6,2023-05-25,1.660.0000.000000 - Transferências do FNAS As...,41.930.974/0001-51/P R COELHO ALIMENTOS LTDA,5 - FUNDO MUN. DE ASSISTENCIA SOCIAL - FMAS,13 - FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,8 - ASSISTÊNCIA SOCIAL,244 - ASSISTÊNCIA COMUNITÁRIA,10316 - ASSISTÊNCIA SOCIAL,2 - SERVIÇOS DE PROTEÇÃO SOCIAL BASICA (PSB),05.13.08.244.10316.2.339030,...,"R$ 0,00","R$ 4.400,00",10562022,10 - DISPENSA DE LICITAÇÃO,312022,Emissão,33903007 - GENEROS DE ALIMENTACAO,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM AQUI...
7,2023-05-25,1.500.0000.000000 - Impostos não vinculados,27.490.577/0001-85/LEANDRO MATERIAIS PARA CON...,3 - PREFEITURA MUN.

In [30]:
driver.quit() # Fechando o navegador

## Passo 3: Limpar a base

Resetando o 'index'

In [31]:
df.reset_index(inplace = True, drop = True)
df

,data,fonte,fornecedor,orgao_unidade_gestora,unidade,funcao,subfuncao,programa,acao,dotacao_orcamentaria,...,valor_da_anulacao_da_ordem_de_pagamento,valor_total_do_pagamento,processo_administrativo,modalidade,licitacao_ano,fase,rubrica_despesa,numero_do_contrato,tipo_do_empenho,bem_fornecido_ou_servico_prestado
0,2023-05-31,1.500.1002.000000 - ASPS - Ações e Serviços P...,24.074.470/0001-68/CLINICA MEDICA CAMPOS EIRE...,4 - FUNDO MUN. DE SAUDE FORMOSO DO ARAGUAIA,12 - FUNDO MUNICIPAL DE SAUDE,10 - SAÚDE,302 - ASSISTÊNCIA HOSPITALAR E AMBULATORIAL,10315 - SAÚDE DIREITO DO CIDADÃO,2 - MANTER PRONTO SOCORRO / CAPS,04.12.10.302.10315.2.339039,...,"R$ 0,00","R$ 0,00",722021,19 - Chamamento/Credenciamento Público,12021,Emissão,33903950 - MÉDICOS,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM CRED...
1,2023-05-30,1.500.0000.000000 - Impostos não vinculados,38.480.650/0001-54/CHARLES ALVES PEREIRA 0162...,3 - PREFEITURA MUN. DE FORMOSO DO ARAGUAIA,"3 - SECRETARIA MUNICIPAL DE TURISMO, ESPORTE",27 - DESPORTO E LAZER,812 - DESPORTO COMUNITÁRIO,"10337 - Incentivo ao Esporte, Lazer e Turismo",2 - MANTER ESPAÇOS ESPORTIVOS,03.03.27.812.10337.2.339030,...,"R$ 0,00","R$ 270,00",5972023,99 - NÃO SE APLICA LICITAÇÃO,,Emissão,33903024 - MATERIAL PARA MANUTENCAO DE BENS ...,,Ordinário,EMPENHO EMITIDO PARA OCORRRER DESPESA COM AQU...
2,2023-05-30,1.500.1002.000000 - ASPS - Ações e Serviços P...,07.626.776/0001-60/CIRURGICA SAO FELIPE PRODU...,4 - FUNDO MUN. DE SAUDE FORMOSO DO ARAGUAIA,12 - FUNDO MUNICIPAL DE SAUDE,10 - SAÚDE,301 - ATENÇÃO BÁSICA,10315 - SAÚDE DIREITO DO CIDADÃO,2 - ADQUIRIR MEDICAMENTOS E MATERIAIS PARA U,04.12.10.301.10315.2.339030,...,"R$ 0,00","R$ 16.739,24",962022,20 - ADESÃO À ATA DE REGISTRO DE PREÇOS,222022,Emissão,33903036 - MATERIAL HOSPITALAR,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM ADE...
3,2023-05-26,1.660.0000.000000 - Transferências do FNAS As...,34.223.365/0001-05/L.P. SIQUEIRA,5 - FUNDO MUN. DE ASSISTENCIA SOCIAL - FMAS,13 - FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,8 - ASSISTÊNCIA SOCIAL,244 - ASSISTÊNCIA COMUNITÁRIA,10316 - ASSISTÊNCIA SOCIAL,2 - PROMOVER APRIMORAMENTO GESTÃO DO CADRAST,05.13.08.244.10316.2.339030,...,"R$ 0,00","R$ 2.335,26",14072021,14 - PREGÃO ELETRÔNICO,272021,Emissão,33903024 - MATERIAL PARA MANUTENCAO DE BENS ...,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM AQUI...
4,2023-05-25,1.500.0000.000000 - Impostos não vinculados,25.043.514/0001-55/SECRETARIA DA FAZENDA,3 - PREFEITURA MUN. DE FORMOSO DO ARAGUAIA,9 - SECRETARIA MUNICIPAL DE INFRA-ESTRUTURA,15 - URBANISMO,451 - INFRA-ESTRUTURA URBANA,10312 - INFRA-ESTRUTURA PÚBLICA,2 - MANTER A MALHA VIARIA URBANAS,03.09.15.451.10312.2.339047,...,"R$ 0,00","R$ 72,11",10632023,99 - NÃO SE APLICA LICITAÇÃO,,Emissão,33904710 - TAXAS,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM PAGA...
5,2023-05-25,1.660.0000.000000 - Transferências do FNAS As...,41.930.974/0001-51/P R COELHO ALIMENTOS LTDA,5 - FUNDO MUN. DE ASSISTENCIA SOCIAL - FMAS,13 - FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,8 - ASSISTÊNCIA SOCIAL,244 - ASSISTÊNCIA COMUNITÁRIA,10316 - ASSISTÊNCIA SOCIAL,2 - PROMOVER APRIMORAMENTO GESTÃO DO CADRAST,05.13.08.244.10316.2.339030,...,"R$ 0,00","R$ 4.609,93",10562022,10 - DISPENSA DE LICITAÇÃO,312022,Emissão,33903007 - GENEROS DE ALIMENTACAO,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM AQUI...
6,2023-05-25,1.660.0000.000000 - Transferências do FNAS As...,41.930.974/0001-51/P R COELHO ALIMENTOS LTDA,5 - FUNDO MUN. DE ASSISTENCIA SOCIAL - FMAS,13 - FUNDO MUNICIPAL DE ASSISTENCIA SOCIAL,8 - ASSISTÊNCIA SOCIAL,244 - ASSISTÊNCIA COMUNITÁRIA,10316 - ASSISTÊNCIA SOCIAL,2 - SERVIÇOS DE PROTEÇÃO SOCIAL BASICA (PSB),05.13.08.244.10316.2.339030,...,"R$ 0,00","R$ 4.400,00",10562022,10 - DISPENSA DE LICITAÇÃO,312022,Emissão,33903007 - GENEROS DE ALIMENTACAO,,Ordinário,EMPENHO EMITIDO PARA OCORRER DESPESA COM AQUI...
7,2023-05-25,1.500.0000.000000 - Impostos não vinculados,27.490.577/0001-85/LEANDRO MATERIAIS PARA CON...,3 - PREFEITURA MUN.

Verificando o tipo das variáveis p/ realizar eventuais conversões.

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 28 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   data                                     30 non-null     datetime64[ns]
 1   fonte                                    30 non-null     object        
 2   fornecedor                               30 non-null     object        
 3   orgao_unidade_gestora                    30 non-null     object        
 4   unidade                                  30 non-null     object        
 5   funcao                                   30 non-null     object        
 6   subfuncao                                30 non-null     object        
 7   programa                                 30 non-null     object        
 8   acao                                     30 non-null     object        
 9   dotacao_orcamentaria                     30 n

Criando uma função p/ retirar o cifrão das variáveis de valor monetário.

In [36]:
def strip_cifrao(text):
    text = re.sub('[^@$!?&.#0-9a-zA-Z_-]',".", text.replace('.', ''))[4:-1]
    text = text.lstrip()
    return text

In [43]:
strip_cifrao(df.valor_total_do_pagamento[2])

'16739.24'

## Passo 4: Salvando a base

In [58]:
df.to_csv('C:\\Users\\guhhh\\OneDrive\\Área de Trabalho\\Workspace\\case_clear_fgv\\df.csv', index=False)